In [1]:
!pip install keras_cv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 33.5 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from keras_cv.models import StableDiffusion
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend


In [3]:
# Initialize the Stable Diffusion model
model_diffusion = StableDiffusion(img_width=512, img_height=512)

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [4]:
# Load a pre-trained MobileNetV2 model for image classification
model_classification = tf.keras.applications.MobileNetV2(weights='imagenet')

14536120/14536120 [==============================] - 0s 0us/step


In [5]:
def generate_images(prompt, batch_size=3):
    # Generate images based on the prompt
    images = model_diffusion.text_to_image(prompt, batch_size=batch_size)
    return images


In [6]:
def plot_images(images):
    # Plot the generated images
    plt.figure(figsize=(20, 20))
    for i, image in enumerate(images):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(image)
        plt.axis("off")
    plt.show()

In [7]:
def preprocess_images(images):
    # Resize images to match the input shape expected by MobileNetV2 (224x224)
    resized_images = [tf.image.resize(img, (224, 224)) for img in images]
    # Preprocess images for MobileNetV2
    preprocessed_images = [tf.keras.applications.mobilenet_v2.preprocess_input(img) for img in resized_images]
    preprocessed_images = np.array(preprocessed_images)
    return preprocessed_images

In [8]:
def classify_images(images, model):
    # Preprocess and resize images
    preprocessed_images = preprocess_images(images)

    # Make predictions
    predictions = model.predict(preprocessed_images)

    # Decode predictions
    decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions)

    return decoded_predictions

In [9]:
def filter_non_relevant_images(images, model):
    # Implement a binary classifier for demonstration

    relevant_images = []
    for image in images:
        # Use the MobileNetV2 model for binary classification
        preprocessed_image = preprocess_images([image])
        prediction = model.predict(preprocessed_image)

        # Modify this condition based on your binary classification model
        if prediction[0][0] > 0.5:
            relevant_images.append(image)

    return relevant_images

In [10]:
# Example usage
prompt = "Headphones"
batch_size = 3

In [ ]:
# Generate and plot images
generated_images = generate_images(prompt, batch_size=batch_size)
relevant_images = filter_non_relevant_images(generated_images, model_classification)
plot_images(generated_images)

26/50 [==============>...............] - ETA: 49:26

In [ ]:
# Classify generated images
classifications = classify_images(generated_images, model_classification)

In [ ]:
# Display the classifications
for i, img_class in enumerate(classifications):
    print(f"Image {i + 1} classification:")
    for j, (imagenet_id, label, score) in enumerate(img_class):
        print(f"{j + 1}: {label} ({score:.2f})")
    print()